<a href="https://colab.research.google.com/github/AlejandroBeltranA/OCVED-ML/blob/master/OCVED_GSR_Trained_v2_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text Classification for OCVED

This script details the process for generating the models used in "Enhancing the Detection of Criminal Organizations in Mexico Using ML and NLP" available at https://www.ocved.mx/

This is 1 of 4 scripts describing this process. In this script we clean the text, use gold standard records of reclassified articles by annotators, build the machine learning models excluding the transformer models, and save the model used to classify the universe of articles.


The below script uses training data generated by student RA's to classify whether an article is relevant to the OCVED project or not. The students selected based on mentions of cartel related violence. It includes mentions of drug seizures, shootouts, and other cartel related events. 


In [ ]:
#Mount google drive
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
# Set cd
%cd /content/drive/
!ls
#Install tqdm package for tracking progress
!pip install tqdm


/content/drive
'My Drive'


# Section 1: Load and Clean 

There are three datasets used in this project. 

The first is "OCVED_2010-2018.csv" which containts 30,842 articles that students classified using the Brat annotation software. Students classified as "Accept" or "Reject" based on a list of paramters that distinguish between articles related to drug trafficking organizations, drugs, and government in Mexico. Articles accepted must have included a mention of an event associated with a dto. More information available at the project website. These articles were scraped from an online news repository and include a large collection of non-relevant articles, that is why we implemented the text classification step. These articles were collected from regional newspapers or non-national sources. 

The second is "National_OCVED.csv" which is a different set of articles that were manually downloaded by research assitants. These articles were identified as relevant at the source, and does not include non-relevant articles. This collection contains 29,995 articles collected from major newspapers in Mexico at the national level from 2000 through 2018. 

The third is "correct_classification.csv" which is a subset of 1 & 2 combined. In a previous iteration we identified 7,500 articles were the model and annotators did not agree. To increase the accuracy of the final model we implemented a human in the loop approach where these articles were reclassified by the annotators to confirm the correct classification. They were thus reviewed at the initial stage by the annotator, at a second stage by the machine, and a third stage again by the annotator. In addition, if the annotator was uncertain of the final classification these were reviewed by the researchers to confirm the correct classification. Given the multiple reviews and attention to these articles we classify them as "Gold Standard of Record" articles. 

In [ ]:
# Packages used in this section.
import glob
import subprocess
import os
import pandas as pd

First dataset. file_id contains the date of publication. label is the assigned tag. Excerpt is a subset of text annotated by students. Text is the full article.  

In [ ]:
df = pd.read_csv('My Drive/Data/OCVED/BRAT/OCVED_2010-2018.csv')
df.head()


,file_id,label,excerpt,text
0,20160926__550828948.txt,Reject,Reject 127 492\tCIUDAD DE MÉXICO (El Universal...,"\nSeptember 26, 2016\n|\nPublication: \n ..."
1,20161013__552467359.txt,Reject,Reject 125 341\tNUEVA YORK (EFE). Michael Jack...,"\nOctober 13, 2016\n|\nPublication: \n ..."
2,20160209__518137112.txt,Reject,"Reject 155 384\tEn enero de 2016, los precios ...","\nFebruary 09, 2016 (10:29)\n|\nPublication: \..."
3,20160817__546311869.txt,Reject,Reject 136 394\t20160817.-El centro comercial ...,"\nAugust 17, 2016\n|\nPublication: \n ..."
4,20161129__557525748.txt,Reject,Reject 137 379\tEl alcalde Mauricio Vila Dosal...,"\nNovember 29, 2016 (03:00)\n|\nPublication: \..."


Count the number of accept & reject tags. The data is unbalanced towards reject. 

In [ ]:
count = df.groupby('label')['label'].count()
count

label
Accept     7183
Reject    23659
Name: label, dtype: int64

Dataset 2 is added here. file_id contains the data, label is the classification, and text is the full article. 

In [ ]:
nat = pd.read_csv('My Drive/Data/OCVED/National/txt_docs/National_OCVED.csv')
nat.head()

,date,file_id,label,text
0,5 de Enero de 2000 \nTranslation powered by Go...,20000105001_NAC.txt,Accept,"\n es \n \n \n Las Margaritas, Chis., 5 Ene (N..."
1,4 de Enero de 2000 \nTranslation powered by Go...,20000105002_NAC.txt,Accept,"\n es \n \n \n México, 4 Ene (NTX).- La Policí..."
2,6 de Enero de 2000 \nTranslation powered by Go...,20000106001_NAC.txt,Accept,"\n es \n \n \n México, 6 Ene (NTX).- Elementos..."
3,6 de Enero de 2000 \nTranslation powered by Go...,20000106002_NAC.txt,Accept,"\n es \n \n \n Monterrey, NL., 6 Ene (NTX).- L..."
4,6 de Enero de 2000 \nTranslation powered by Go...,20000106003_NAC.txt,Accept,"\n es \n \n \n México, 6 Ene (NTX).- Elementos..."


Combine the two data sets.

In [ ]:
data = []
data.append(df)
data.append(nat)
frame = pd.concat(data, axis=0, ignore_index=True, sort=True).sort_values('file_id', ascending= True).drop(columns=['date', 'excerpt'])
frame.head()

,file_id,label,text
30842,20000105001_NAC.txt,Accept,"\n es \n \n \n Las Margaritas, Chis., 5 Ene (N..."
30843,20000105002_NAC.txt,Accept,"\n es \n \n \n México, 4 Ene (NTX).- La Policí..."
30844,20000106001_NAC.txt,Accept,"\n es \n \n \n México, 6 Ene (NTX).- Elementos..."
30845,20000106002_NAC.txt,Accept,"\n es \n \n \n Monterrey, NL., 6 Ene (NTX).- L..."
30846,20000106003_NAC.txt,Accept,"\n es \n \n \n México, 6 Ene (NTX).- Elementos..."


We have a total of 60,837 articles collected between the subnational dataset and the national media dataset. The combined categories favor the Accept classification now. 

In [ ]:
count = frame.groupby('label')['label'].count()
count

label
Accept    37178
Reject    23659
Name: label, dtype: int64

Now let's add the GSR labels. 


In [ ]:
gsr = pd.read_csv("My Drive/Data/OCVED/Classifier/predictions_v2/correct_classification.csv")
gsr.head()

,file_id,correct
0,20000105001_NAC.txt,1
1,20000105002_NAC.txt,1
2,20000106001_NAC.txt,1
3,20000106002_NAC.txt,1
4,20000106003_NAC.txt,1


Here we add th GSR labels to the full dataset.

In [ ]:
data = pd.merge(frame, gsr)
data.head()

,file_id,label,text,correct
0,20000105001_NAC.txt,Accept,"\n es \n \n \n Las Margaritas, Chis., 5 Ene (N...",1
1,20000105002_NAC.txt,Accept,"\n es \n \n \n México, 4 Ene (NTX).- La Policí...",1
2,20000106001_NAC.txt,Accept,"\n es \n \n \n México, 6 Ene (NTX).- Elementos...",1
3,20000106002_NAC.txt,Accept,"\n es \n \n \n Monterrey, NL., 6 Ene (NTX).- L...",1
4,20000106003_NAC.txt,Accept,"\n es \n \n \n México, 6 Ene (NTX).- Elementos...",1


Now it's time to clean the text. The example below shows the original structure of the text. There is html code we need to clean out, stop words, accents, and other noise.

In [ ]:
data['text'][1]

'\n es \n \n \n México, 4 Ene (NTX).- La Policía Federal Preventiva (PFP) informó que en las últimas 24 horas detuvo a ocho individuos, quienes habían asaltado un camión de pasajeros a mano armada, en las inmediaciones del puerto de Mazatlán, Sinaloa. \n De acuerdo con el reporte más reciente de esta corporación, también en Nogales, Sonora, se decomisaron 535 kilogramos de mariguana, al detener a una camioneta Suburban Chevrolet por haber cometido una infracción. \n La PFP precisó que en el caso del asalto al omnibus, propiedad de Transportes Turistar, el conductor Leonardo Nicandro Contreras denunció que a la altura del kilometro 321, de la carretera atamoros-Mazatlán, cuatro individuos portando un arma corta y dos largas detuvieron la unidad para robarlos. \n Contreras dijo que los asaltantes llevaban el rostro cubierto con pasamontañas y despojaron de sus pertenencias y dinero, por un monto aproximado de 12 mil 600 pesos y 350 dólares, a los pasajeros. \n De inmediato se procedió a 

Additional packages for cleaning out the text and starting the machine learning pipeline.

In [ ]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
#from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import xgboost
from sklearn import model_selection, naive_bayes, svm, linear_model, decomposition, ensemble
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, precision_score, recall_score, f1_score

We use the Spacy libraries Spanish language lemmatizer to reduce words to their lemma, reducing the size of the overall dictionary further down the pipeline. 

In [ ]:
%%capture
!pip install es-lemmatizer
!pip install -U spacy
!sudo python -m spacy download es_core_news_sm

# Download stopwords dictionary from nltk
import re
import nltk
nltk.download('stopwords')

In [ ]:
np.random.seed(1000)

Code below cleans out accents and normalizes text.



In [ ]:
import unicodedata
import string
# BEGIN SHAVE_MARKS_LATIN
def shave_marks_latin(txt):
    """Remove all diacritic marks from Latin base characters"""
    norm_txt = unicodedata.normalize('NFD', txt)  # <1>
    latin_base = False
    keepers = []
    for c in norm_txt:
        if unicodedata.combining(c) and latin_base:   # <2>
            continue  # ignore diacritic on Latin base char
        keepers.append(c)                             # <3>
        # if it isn't combining char, it's a new base char
        if not unicodedata.combining(c):              # <4>
            latin_base = c in string.ascii_letters
    shaved = ''.join(keepers)
    return unicodedata.normalize('NFC', shaved)   # <5>
# END SHAVE_MARKS_LATIN
def shave_marks(txt):
    """Remove all diacritic marks"""
    norm_txt = unicodedata.normalize('NFD', txt)  # <1>
    shaved = ''.join(c for c in norm_txt
                     if not unicodedata.combining(c))  # <2>
    return unicodedata.normalize('NFC', shaved)  # <3>
# END SHAVE_MARKS

Now we load the spacy nlp module, and add the lemmatizer pipe and POS tagger. 

In [ ]:
from es_lemmatizer import lemmatize
import es_core_news_sm

nlp = es_core_news_sm.load()
nlp.add_pipe(lemmatize, after="tagger")

Below we load the Spanish stopwords dictionary from nltk and we include additional stopwords that generate noise within the documents. By eliminating these recurring words we can make sure the tf-idf dictionary contains relevant terms. 


In [ ]:
from nltk.corpus import stopwords

##Creating a list of stop words and adding custom stopwords
stop_words = set(stopwords.words("spanish"))
##Creating a list of custom stopwords
new_words = ["daily", "newspaper", "reforma", "publication", "universal", "iv", "one", "two", "august" , "excelsior", "online",
             "november", "july", "september", "june", "october", "december", "print", "edition", "news", "milenio", "january", "international",
             "march", "april", "july", "february", "may", "october", "el occidental", "comments", "powered", "display", "space", 
             "javascript", "trackpageview", "enablelinktracking", "location", "protocol", "weboperations", "settrackerurl", "left", 
             "setsiteid", "createelement", "getelementsbytagname", "parentnode", "insertbefore", "writeposttexto", "everykey", "passwords"
             "writecolumnaderechanotas", "anteriorsiguente", "anteriorsiguiente", "writefooter", "align", "googletag", "writeaddthis", "writefooteroem", 
             "diario delicias", "diario tampico", "the associated press", "redaccion" , "national", "diario yucatan", "mural", "periodico", 
             "new", "previously", "shown" , "a", "para", "tener" , "haber", "ser" , "mexico city", "states", "city", "and", "elsolde", "recomendamos", 
            "diario chihuahua" , "diario juarez" , "el norte", "voz frontera" , "regional" , "de"  , "el sol" , "el" , "sudcaliforniano" , "washington",
            "union morelos", "milenio" , "notimex", "el financiero" , "financiero" , "forum magazine" , "economista" , "gmail" , "financial", "el" , "de",
             "la", "del", "de+el" , "a+el" , "shortcode" , "caption", "cfcfcf", "float", "item", "width", "follow", "aaannnnn", "gmannnnn", 
             "dslnnnnn", "jtjnnnnn", "lcgnnnnn", "jgcnnnnn", "vhannnnn",  "mtc", "eleconomista", "monitoreoif", "infosel", "gallery", 
             "heaven", "div", "push" , "translate", "google"]
stop_words = stop_words.union(new_words)
stop_words = shave_marks(repr(stop_words))

The below command builds the corpus. It removes punctuations, tags, special characters, white space, and then extracts the lemma of each word, then removes accents. 

In [ ]:
dataset = data

In [ ]:
corpus = []
for i in (dataset.itertuples()): #tqdm
    text = shave_marks_latin(i.text)
    #Remove punctuations
    text = re.sub('[^a-zA-Z]', ' ', text)
    #Convert to lowercase
    #text = shave_marks_latin(text)
    #text = text.lower()
    #remove tags
    text=re.sub("&lt;/?.*?&gt;"," &lt;&gt; ",text)
    # remove special characters and digits
    text=re.sub("(\\d|\\W)+"," ",text)
    text = re.sub(' +', ' ', text)
    #Lemmatisation
    doc = nlp(text)
    text = [token.lemma_ for token in doc if token.lemma_ not in stop_words] 
    text = " ".join(text)
    text = shave_marks(text)
    label = i.correct
    file_id = i.file_id
    corpus.append({ 'text': text,  'label': label  , 'file_id': file_id })
print ("done")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  


The cleaning process can take over an hour on this dataset. I tried speeding it up using mapping instead but the lemmatization was being inconsistent. So instead I use a regular loop, it is easier to see which articles cause trouble with this method. 


In [ ]:
data = pd.DataFrame(corpus)
data.head()

Given how long it takes to preprocess the text, we save it into a csv for pulling it in later in other scripts. 

In [ ]:
#data.to_csv('/content/drive/My Drive/Data/OCVED/Classifier/universe/preprocessed_text.csv')

In [ ]:
data = pd.read_csv('/content/drive/My Drive/Data/OCVED/Classifier/universe/preprocessed_text.csv')
data = data[["file_id", "text"]]
data.head()

,file_id,text
0,20000105001_NAC.txt,margaritas chis ntx elementos ejercito exicano...
1,20000105002_NAC.txt,ntx policia federal preventiva pfp informo ult...
2,20000106001_NAC.txt,ntx elementos policia judicial federal pjf ase...
3,20000106002_NAC.txt,monterrey ntx policia ministerial reporto homb...
4,20000106003_NAC.txt,ntx elementos policia judicial federal pjf ase...


Assign the GSR labels for safe measure. 

In [ ]:
data = pd.merge(data, gsr, on = "file_id").rename(columns= {"correct":"label"})
data.head()

,file_id,text,label
0,20000105001_NAC.txt,margaritas chis ntx elementos ejercito exicano...,1
1,20000105002_NAC.txt,ntx policia federal preventiva pfp informo ult...,1
2,20000106001_NAC.txt,ntx elementos policia judicial federal pjf ase...,1
3,20000106002_NAC.txt,monterrey ntx policia ministerial reporto homb...,1
4,20000106003_NAC.txt,ntx elementos policia judicial federal pjf ase...,1


# Machine Learning Text Classifier

In this section we use the 5 most common algorithms for text classification. Naive-Bayes, Logistic Regression, Support-Vector Machines, Random Forest, and Extreme Gradient Boosting. For each model, two strategies are used to evalaute their performance: The first uses 90% of training data and 10% for the validation set, this allows the model to learn from a larger subsample of data. The second uses a k-fold cross validation approach, where we split the data into 5 different folds, and train 4 different models on this data, using the next fold as the validation set. This produces multiple performance scores that demosntrate how well the model performs on smaller subsets of data. The paper reports both.

The first step is to convert our text into a numerical statistic that weighs the frequency of each word across articles. For this paper we chose term frequency-inverse document frequency. The TfidfVectorizer command automatically calculates these weights, here we cap the number of words to use at 5,000. Earlier we reduced each word to its lemma, this allows the tf-idf to weigh words like "asaltaron" the same as "asaltar" thus allowing more room for a more diverse dictionary. Tf-idf helps reflect how important a word is in the context of the entire document and within the whole collection of documents. These weights are reflected in a numeric matrix for each article. 

In [ ]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(data['text'])
X = Tfidf_vect.transform(data['text'])
Y = data['label']

In [ ]:
Encoder = LabelEncoder()
Y = Encoder.fit_transform(data['label']) # confirm this works 

This is what article 1 above looks like after being transformed.

In [ ]:
print(X[1])

  (0, 4998)	0.12696455339208565
  (0, 4875)	0.11085207796774008
  (0, 4749)	0.10904631456121533
  (0, 4678)	0.07483627627218864
  (0, 4464)	0.06786308207076301
  (0, 4359)	0.11022809036411535
  (0, 4333)	0.07393382056432098
  (0, 4294)	0.09574256488986035
  (0, 4153)	0.06905832872132098
  (0, 3992)	0.04072028376000164
  (0, 3963)	0.10006890937100248
  (0, 3911)	0.08252630430208133
  (0, 3779)	0.11324146113609339
  (0, 3775)	0.06379221646792467
  (0, 3755)	0.08766514068672171
  (0, 3705)	0.030600413300888477
  (0, 3694)	0.0702770454357659
  (0, 3676)	0.2870360207784892
  (0, 3675)	0.10316416354605946
  (0, 3642)	0.0772596523524065
  (0, 3608)	0.033560741914438606
  (0, 3578)	0.07135467001391413
  (0, 3497)	0.06333100647826001
  (0, 3389)	0.09254105791792451
  (0, 3307)	0.13452900522902628
  :	:
  (0, 1721)	0.05384101980644171
  (0, 1547)	0.05430208803402592
  (0, 1527)	0.039632525457918566
  (0, 1517)	0.2500660168350958
  (0, 1516)	0.09276197108669401
  (0, 1498)	0.09030476378424362
  (

Now we split training and test data for the first estimation method. Here 90% of articles are used as training data and 10% are held out for testing. 

In [ ]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(X,Y,test_size=0.10) #,test_size=0.1

Let's confirm how many of each label are in the training and test data sets. feel free to play around with this. 

In [ ]:
Train_Y
unique, counts = np.unique(Test_Y, return_counts=True)
dict(zip(unique, counts))
#unique, counts = np.unique(Train_Y, return_counts=True)
#dict(zip(unique, counts))

{0: 2512, 1: 3569}

This is the first algorithm. This is a Naive Bayes classifier. It will print out the accuracy score of the algorithm based on the training data TFIDF on the test data. We also print other measures of performance for reference, for the paper we use F1 across all models. 

In [ ]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X,Train_Y)
# predict the labels on validation dataset
predictions_NB = Naive.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(Test_Y, predictions_NB)*100)
print("Naive Bayes Precision Score -> ",precision_score( Test_Y, predictions_NB) *100)
print("Naive Bayes Recall Score -> ",recall_score( Test_Y, predictions_NB) *100)
print("Naive Bayes F1 Score -> ",f1_score( Test_Y, predictions_NB) *100)

print("Naive Bayes Precision, Recall, and F1 by Label -> ",precision_recall_fscore_support( Test_Y, predictions_NB))

Naive Bayes Accuracy Score ->  89.3603025818122
Naive Bayes Precision Score ->  87.55784061696657
Naive Bayes Recall Score ->  95.43289436817035
Naive Bayes F1 Score ->  91.32591500201099
Naive Bayes Precision, Recall, and F1 by Label ->  (array([0.92560475, 0.87557841]), array([0.80732484, 0.95432894]), array([0.86242824, 0.91325915]), array([2512, 3569]))


Next is the cross validation for the NB model. First, we create a dataframe to store the scores for building out the tables in our paper. 

In [ ]:
import pandas as pd
saved_scores = pd.DataFrame()

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import ShuffleSplit
import pandas as pd
# This is where we specify how many folds to use. 5 are specified with 10% left out for final validation
cv = ShuffleSplit(n_splits=5, test_size=0.1, random_state=1000)

# cross-validate
scores = cross_val_score(Naive,
                        X,
                        Y,
                        cv=cv,
                         scoring='f1') # Calculating F1 score
saved_scores["NB_tfidf"] = scores
print(saved_scores["NB_tfidf"])
print("Average F1 across folds:" ,saved_scores["NB_tfidf"].mean())

0    0.912360
1    0.914939
2    0.911957
3    0.916314
4    0.918840
Name: NB_tfidf, dtype: float64
Average F1 across folds: 0.9148819254082501


The NB model performs fairly similar across folds as compared to the more complete model above. 

The below uses a support vector machine (SVM) classifier. Same as above, it prints variosu performance scores for how well the training algorithm perfoms using 90% of articles on the test data. 

In [ ]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.LinearSVC(C=1.0)
SVM.fit(Train_X,Train_Y)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)
print("SVM Precision Score -> ",precision_score( Test_Y, predictions_SVM) *100)
print("SVM Recall Score -> ",recall_score( Test_Y, predictions_SVM) *100)
print("SVM F1 Score -> ",f1_score( Test_Y, predictions_SVM) *100)

print("SVM Precision, Recall, and F1 by Label -> ",precision_recall_fscore_support( Test_Y, predictions_SVM))

SVM Accuracy Score ->  93.60302581812202
SVM Precision Score ->  94.04432132963989
SVM Recall Score ->  95.12468478565424
SVM F1 Score ->  94.58141802479454
SVM Precision, Recall, and F1 by Label ->  (array([0.92958316, 0.94044321]), array([0.91441083, 0.95124685]), array([0.92193458, 0.94581418]), array([2512, 3569]))


This model performs really well! Let's see how well it does across folds.

In [ ]:
# cross-validate
scores = cross_val_score(SVM,
                        X,
                        Y,
                        cv=cv,
                         scoring='f1')
saved_scores["SVM_tfidf"] = scores
print(saved_scores["SVM_tfidf"])
print("Average F1 across folds:" ,saved_scores["SVM_tfidf"].mean())

0    0.951697
1    0.944383
2    0.950404
3    0.948476
4    0.946773
Name: SVM_tfidf, dtype: float64
Average F1 across folds: 0.9483468431570617


The SVM is learning the feautres of our data really well. Even across folds it is averaging .94 F1. 

Next we used a simple logistic regression. Again we are printing out different performance scores using 90% training data.

In [ ]:
# Classifier - Algorithm - Logistic Regression
# fit the training dataset on the classifier
Linear = linear_model.LogisticRegression()
Linear.fit(Train_X,Train_Y)
# predict the labels on validation dataset
predictions_Linear = Linear.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("Logistic Regression Accuracy Score -> ",accuracy_score(predictions_Linear, Test_Y)*100)

print("Logistic Regression Precision Score -> ",precision_score( Test_Y, predictions_Linear) *100)
print("Logistic Regression Recall Score -> ",recall_score( Test_Y, predictions_Linear) *100)
print("Logistic Regression F1 Score -> ",f1_score( Test_Y, predictions_Linear) *100)

print("Logistic Regression Precision, Recall, and F1 by Label -> ",precision_recall_fscore_support( Test_Y, predictions_Linear))

Logistic Regression Accuracy Score ->  93.52080249958888
Logistic Regression Precision Score ->  93.57672248147132
Logistic Regression Recall Score ->  95.51695152703839
Logistic Regression F1 Score ->  94.53688297282307
Logistic Regression Precision, Recall, and F1 by Label ->  (array([0.93437244, 0.93576722]), array([0.90684713, 0.95516952]), array([0.92040404, 0.94536883]), array([2512, 3569]))


Let's see how well it does across folds. 

In [ ]:
# cross-validate
scores = cross_val_score(Linear,
                        X,
                        Y,
                        cv=cv,
                         scoring='f1')
saved_scores["LR_tfidf"] = scores
print(saved_scores["LR_tfidf"])
print("Average F1 across folds:" ,saved_scores["LR_tfidf"].mean())

0    0.949662
1    0.949437
2    0.947398
3    0.950157
4    0.949925
Name: LR_tfidf, dtype: float64
Average F1 across folds: 0.9493155995636389


Again averaging a really good .94 F1. 

Next we try a couple of ensemble methods. The first is a random forest classifier using the 90% training data. 

In [ ]:

# Classifier - Algorithm - Random Forest
# fit the training dataset on the classifier
RForest = ensemble.RandomForestClassifier()
RForest.fit(Train_X,Train_Y)
# predict the labels on validation dataset
predictions_RForest = RForest.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("Random Forest Accuracy Score -> ",accuracy_score(predictions_RForest, Test_Y)*100)

print("Random Forest Precision Score -> ",precision_score( Test_Y, predictions_RForest) *100)
print("Random Forest Recall Score -> ",recall_score( Test_Y, predictions_RForest) *100)
print("Random Forest F1 Score -> ",f1_score( Test_Y, predictions_RForest) *100)

print("Random Forest Precision, Recall, and F1 by Label -> ",precision_recall_fscore_support( Test_Y, predictions_RForest))

Random Forest Accuracy Score ->  91.0541029435948
Random Forest Precision Score ->  88.93178893178893
Random Forest Recall Score ->  96.80582796301485
Random Forest F1 Score ->  92.7019050174403
Random Forest Precision, Recall, and F1 by Label ->  (array([0.94808743, 0.88931789]), array([0.82882166, 0.96805828]), array([0.884452  , 0.92701905]), array([2512, 3569]))


It does not do as well as the linear models, but still great performance. Let's see how well it does across folds. 

In [ ]:
# cross-validate
scores = cross_val_score(RForest,
                        X,
                        Y,
                        cv=cv,
                         scoring='f1')
saved_scores["RF_tfidf"] = scores
print(saved_scores["RF_tfidf"])
print("Average F1 across folds:" ,saved_scores["RF_tfidf"].mean())

0    0.929736
1    0.928249
2    0.926358
3    0.928036
4    0.930701
Name: RF_tfidf, dtype: float64
Average F1 across folds: 0.9286158236834702


Finally, we run an extreme gradient boosting model. The first using the 90% training data. 

In [ ]:
# Classifier - Algorithm - Extereme Gradient Boosting
# fit the training dataset on the classifier
xgboost = xgboost.XGBClassifier()
xgboost.fit(Train_X,Train_Y)
# predict the labels on validation dataset
predictions_XGBoost = xgboost.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("Extereme Gradient Boosting Accuracy Score -> ",accuracy_score(predictions_XGBoost, Test_Y)*100)

print("Extereme Gradient Boosting Precision Score -> ",precision_score( Test_Y, predictions_XGBoost) *100)
print("Extereme Gradient Boosting Recall Score -> ",recall_score( Test_Y, predictions_XGBoost) *100)
print("Extereme Gradient Boosting F1 Score -> ",f1_score( Test_Y, predictions_XGBoost) *100)

print("Extereme Gradient Boosting Precision, Recall, and F1 by Label -> ",precision_recall_fscore_support( Test_Y, predictions_XGBoost))

Extereme Gradient Boosting Accuracy Score ->  87.69939154744286
Extereme Gradient Boosting Precision Score ->  89.0179806362379
Extereme Gradient Boosting Recall Score ->  90.16531241244046
Extereme Gradient Boosting F1 Score ->  89.58797327394208
Extereme Gradient Boosting Precision, Recall, and F1 by Label ->  (array([0.85766423, 0.89017981]), array([0.8419586 , 0.90165312]), array([0.84973885, 0.89587973]), array([2512, 3569]))


Let's compare across folds. 

In [ ]:
# cross-validate
scores = cross_val_score(xgboost,
                        X,
                        Y,
                        cv=cv,
                         scoring='f1')
saved_scores["XGB_tfidf"] = scores
print(saved_scores["XGB_tfidf"])
print("Average F1 across folds:" ,saved_scores["XGB_tfidf"].mean())

0    0.905295
1    0.896752
2    0.901283
3    0.902095
4    0.902365
Name: XGB_tfidf, dtype: float64
Average F1 across folds: 0.9015580215307324


Not as great as the others. 

In [ ]:
saved_scores

,NB_tfidf,SVM_tfidf,LR_tfidf,RF_tfidf,XGB_tfidf
0,0.912360,0.951697,0.949662,0.925926,0.905295
1,0.914939,0.944383,0.949437,0.926887,0.896752
2,0.911957,0.950404,0.947398,0.927948,0.901283
3,0.916314,0.948476,0.950157,0.930104,0.902095
4,0.918840,0.946773,0.949925,0.928704,0.902365


Here we can compare across models. Please note that each time this is run the validation set is different so these numbers may not match up perfectly with the paper. Regardless, after multiple runs the logistic regression consistently out performed all the other models. 

# Topic Modelling

In this next section we run a Latent Dirichlet Allocation model to build topic summaries to better demonstrate the overrall themes captured in our subsample of data. 

Here we switch to counter vectorizer that weighs words within the individual document rather than the whole collection. 

In [ ]:
# create a count vectorizer object 
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(data['text'])

# transform 
xcount =  count_vect.transform(data['text'])


Below I use latent Dirichlet allocation to extract topic summaries for the entire data set. 

In [ ]:
# train a LDA Model
import numpy
lda_model = decomposition.LatentDirichletAllocation(n_components=20, learning_method='online', max_iter=20)
X_topics = lda_model.fit_transform(xcount)
topic_word = lda_model.components_ 
vocab = count_vect.get_feature_names()

# view the topic models
n_top_words = 10
topic_summaries = []
for i, topic_dist in enumerate(topic_word):
    topic_words = numpy.array(vocab)[numpy.argsort(topic_dist)][:-(n_top_words+1):-1]
    topic_summaries.append(' '.join(topic_words))

Below I print out the topic summaries.

In [ ]:
topic_summaries

['peso millon dinero unidos dolar ninos internacional aeropuerto valor mexicano',
 'pgj naucalpan paciente tlalnepantla enfermedad nezahualcoyotl facebook tratamiento personaje alcaldesa',
 'arma fuego cartucho vehiculo asegurar calibrar tres cargador largo calibre',
 'hombre policia colonia calle hora hecho persona lugar encontrar ciudad',
 'decir hacer poder seguridad dar ir municipal parte ciudad realizar',
 'municipio cuerpo encontrar guerrero michoacan comunidad persona localizar estatal cadaver',
 'hallan www recomendar toma imputado edomex temperatura https siglo bloquear',
 'investigacion delito general detener policia federal procuraduria fiscalia presunto persona',
 'gobierno nacional presidente gobernador estatal alcalde partido pais tema local',
 'security justice paq function of document cmd true piwik original',
 'clandestino fosa interno sustancia centro droga penal litro laboratorio penitenciario',
 'nuevo leon tamaulipas monterrey candidato san reynosa coahuila garza l

# Saving the Model

The final step is to save the best performing model for its use on the universe of articles. There is an additional script that uses this training data with transformer models like BERT and another using convolutional neural networks. The logistic regression outperformed the other models which is why we are saving it below for use on the full list of articles. 

Let's save the output of our algorithm as a usable file for the universe of articles. 


First I'm going to save the encoder as a pickle file.

In [ ]:
import pickle


#exporting the departure encoder
output = open('/content/drive/My Drive/Data/OCVED/Classifier/algorithm/OCVED_encoder_v2.pkl', 'wb')
pickle.dump(Encoder, output)
output.close()

Let's test the pickle file, see if it saved correctly. 

In [ ]:
pkl_file = open('/content/drive/My Drive/Data/OCVED/Classifier/algorithm/OCVED_encoder_v2.pkl', 'rb')
encoder = pickle.load(pkl_file) 
pkl_file.close()

valid_encoder = encoder.inverse_transform(Train_Y)
valid_encoder

array([1, 1, 1, ..., 1, 0, 0])

Let's run the LR one more time to confirm it does well. 

In [ ]:
# Classifier - Algorithm - Logistic Regression
# fit the training dataset on the classifier
Linear = linear_model.LogisticRegression()
Linear.fit(Train_X,Train_Y)
# predict the labels on validation dataset
predictions_Linear = Linear.predict(Test_X)
# Use accuracy_score function to get the accuracy
print("Logistic Regression Accuracy Score -> ",accuracy_score(predictions_Linear, Test_Y)*100)

print("Logistic Regression Precision Score -> ",precision_score( Test_Y, predictions_Linear) *100)
print("Logistic Regression Recall Score -> ",recall_score( Test_Y, predictions_Linear) *100)
print("Logistic Regression F1 Score -> ",f1_score( Test_Y, predictions_Linear) *100)

print("Logistic Regression Precision, Recall, and F1 by Label -> ",precision_recall_fscore_support( Test_Y, predictions_Linear))

Logistic Regression Accuracy Score ->  94.0634764019076
Logistic Regression Precision Score ->  94.38386041439476
Logistic Regression Recall Score ->  95.7146806745922
Logistic Regression F1 Score ->  95.044612216884
Logistic Regression Precision, Recall, and F1 by Label ->  (array([0.93576461, 0.9438386 ]), array([0.9163961 , 0.95714681]), array([0.92597909, 0.95044612]), array([2464, 3617]))


In [ ]:
from sklearn.externals import joblib
from sklearn import metrics
# save the model to disk
filename = '/content/drive/My Drive/Data/OCVED/Classifier/algorithm/logistic_model_v2.sav'
joblib.dump(Linear, filename)
 
# some time later...
 
# load the model from disk
loaded_model = joblib.load(filename)
result = loaded_model.score(Test_X_Tfidf, Test_Y)
print(result)



0.9406347640190759


In [ ]:
import pickle
#count_vect.fit(trainDF['text'])

# transform the training and validation data using count vectorizer object
Tfidf_vect_2 =  Tfidf_vect.fit(trainDF['text'])
pickle.dump(Tfidf_vect_2, open("/content/drive/My Drive/Data/OCVED/Classifier/algorithm/Tfidf_vect_3.pickle", "wb"))


In [ ]:
print(len(Tfidf_vect_2.vocabulary_))

5000
